## Summary

Calculate features using [ProtBert](https://github.com/agemagician/ProtTrans).

---

## Imports

In [ ]:
import os
import socket
import sys
import tempfile
from pathlib import Path

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from ev2.plugins.protbert import ProtBert

In [ ]:
pd.set_option("max_columns", 1000)
pd.set_option("max_rows", 1000)

In [ ]:
ProtBert.load_model()

## Parameters

In [ ]:
NOTEBOOK_DIR = Path("02_run_protbert").resolve()
NOTEBOOK_DIR.mkdir(exist_ok=True)

NOTEBOOK_DIR

In [ ]:
if "DATAPKG_OUTPUT_DIR" in os.environ:
    OUTPUT_DIR = Path(os.getenv("DATAPKG_OUTPUT_DIR")).joinpath("elaspic-v2").resolve()
else:
    OUTPUT_DIR = NOTEBOOK_DIR.parent
OUTPUT_DIR.mkdir(exist_ok=True)

OUTPUT_DIR

In [ ]:
if "scinet" in socket.gethostname():
    CPU_COUNT = 40
else:
    CPU_COUNT = max(1, len(os.sched_getaffinity(0)) // 2)

CPU_COUNT

In [ ]:
DATASET_NAME = os.getenv("DATASET_NAME")
TASK_ID = os.getenv("SLURM_ARRAY_TASK_ID")
TASK_COUNT = os.getenv("ORIGINAL_ARRAY_TASK_COUNT") or os.getenv("SLURM_ARRAY_TASK_COUNT")

TASK_ID = int(TASK_ID) if TASK_ID is not None else None
TASK_COUNT = int(TASK_COUNT) if TASK_COUNT is not None else None

DATASET_NAME, TASK_ID, TASK_COUNT

In [ ]:
DEBUG = TASK_ID is None

if DEBUG:
    DATASET_NAME = "elaspic-training-set-core"
    TASK_ID = 87
    TASK_COUNT = 162
else:
    assert DATASET_NAME is not None
    assert TASK_ID is not None
    assert TASK_COUNT is not None

DATASET_NAME, TASK_ID, TASK_COUNT

## Load data

In [ ]:
input_file = OUTPUT_DIR.joinpath("01_load_data", f"{DATASET_NAME}.parquet")

input_file

In [ ]:
pfile = pq.ParquetFile(input_file)

pfile.num_row_groups

In [ ]:
assert TASK_COUNT == pfile.num_row_groups

In [ ]:
INPUT_DF = pfile.read_row_group(TASK_ID - 1).to_pandas(integer_object_nulls=True)

In [ ]:
display(INPUT_DF.head(2))
print(len(INPUT_DF))

In [ ]:
columns = [
    "unique_id",
    "mutations",
]